In [1]:
from openbb import obb
import pandas as pd
import os

In [2]:
os.chdir('..')

In [3]:
#Helper functions for OpenBB data

def get_interest_rate_data():
    """
    Fetches interest rate data for the United States from the OpenBB API.
    Returns:
        dict: A dictionary containing interest rate data for immediate, short, and long durations.
    """
    data = {}
    durations = {'immediate': 'Overnight Interbank Rate', 'short': '3-Month Rate', 'long': '10-Year Rate'}

    for duration, description in durations.items():
        data[description] = obb.economy.interest_rates(
            provider='oecd', 
            country='united_states', 
            duration=duration,
            frequency='monthly'
        )

    return data

def get_cpi_data():
    """
    Fetches Consumer Price Index (CPI) data for the United States from the OpenBB API.
    Returns:
        DataFrame: A DataFrame containing CPI data for the United States.
    """
    cpi_data = obb.economy.cpi(
        country='united_states', 
        transform='yoy',  # Year-over-Year change by default
        provider='oecd'
    )
    return cpi_data

def main():
    # Fetch Interest Rate Data
    print("Fetching Interest Rate Data...")
    interest_rate_data = get_interest_rate_data()
    # for key, value in interest_rate_data.items():
    #     print(f"{key}:\n{value}\n")

    # Fetch CPI Data
    print("Fetching CPI Data...")
    cpi_data = get_cpi_data()
    # print(f"CPI Data:\n{cpi_data}\n")
    return interest_rate_data, cpi_data

    # You can now feed this data to an LLM or perform additional analysis.

In [17]:
from python_scripts.utils import prepare_data_for_simulation

In [6]:
interest_rate_data, cpi_data = main()

Fetching Interest Rate Data...
Fetching CPI Data...


In [7]:
interest_rate_dict = {}

for key in interest_rate_data.keys():
    interest_rate_dict[key] = interest_rate_data[key].to_df()
print(interest_rate_dict)

{'Overnight Interbank Rate':                value        country
date                               
1954-07-01  0.008000  united_states
1954-08-01  0.012200  united_states
1954-09-01  0.010700  united_states
1954-10-01  0.008500  united_states
1954-11-01  0.008300  united_states
...              ...            ...
2024-08-01  0.053300  united_states
2024-09-01  0.048300  united_states
2024-10-01  0.048300  united_states
2024-11-01  0.046383  united_states
2024-12-01  0.044800  united_states

[846 rows x 2 columns], '3-Month Rate':              value        country
date                             
1964-06-01  0.0386  united_states
1964-07-01  0.0387  united_states
1964-08-01  0.0385  united_states
1964-09-01  0.0387  united_states
1964-10-01  0.0394  united_states
...            ...            ...
2024-08-01  0.0512  united_states
2024-09-01  0.0486  united_states
2024-10-01  0.0462  united_states
2024-11-01  0.0453  united_states
2024-12-01  0.0446  united_states

[726 rows x 2 colum

In [8]:
interest_rate_dict

{'Overnight Interbank Rate':                value        country
 date                               
 1954-07-01  0.008000  united_states
 1954-08-01  0.012200  united_states
 1954-09-01  0.010700  united_states
 1954-10-01  0.008500  united_states
 1954-11-01  0.008300  united_states
 ...              ...            ...
 2024-08-01  0.053300  united_states
 2024-09-01  0.048300  united_states
 2024-10-01  0.048300  united_states
 2024-11-01  0.046383  united_states
 2024-12-01  0.044800  united_states
 
 [846 rows x 2 columns],
 '3-Month Rate':              value        country
 date                             
 1964-06-01  0.0386  united_states
 1964-07-01  0.0387  united_states
 1964-08-01  0.0385  united_states
 1964-09-01  0.0387  united_states
 1964-10-01  0.0394  united_states
 ...            ...            ...
 2024-08-01  0.0512  united_states
 2024-09-01  0.0486  united_states
 2024-10-01  0.0462  united_states
 2024-11-01  0.0453  united_states
 2024-12-01  0.0446  united_

In [9]:
cpi_data_df = cpi_data.to_df()
cpi_data_df

,country,value,expenditure
date,,,
1956-01-01,united_states,0.003745,total
1956-02-01,united_states,0.003745,total
1956-03-01,united_states,0.003745,total
1956-04-01,united_states,0.007491,total
1956-05-01,united_states,0.011236,total
...,...,...,...
2024-07-01,united_states,0.028948,total
2024-08-01,united_states,0.025307,total
2024-09-01,united_states,0.024406,total


In [14]:
def clean_df(df,start_date):
    # We don't need all the data, and we also want to resample to hour to match the trade_script
    
    df_copy = df.copy()
    df_copy.index = pd.to_datetime(df_copy.index)
    
    if start_date == 'Max':
        start_date = df_copy.index.max()
        
    df_copy = df_copy[df_copy.index >= start_date]
    return df_copy

In [15]:
cpi_data_df_clean = clean_df(cpi_data_df, 'Max')

In [25]:
for key in interest_rate_dict.keys():
    interest_rate_dict[key] = clean_df(interest_rate_dict[key], 'Max')
print(interest_rate_dict)

{'Overnight Interbank Rate':              value        country
date                             
2024-12-01  0.0448  united_states, '3-Month Rate':              value        country
date                             
2024-12-01  0.0446  united_states, '10-Year Rate':              value        country
date                             
2024-12-01  0.0439  united_states}


In [16]:
cpi_data_df_clean

,country,value,expenditure
date,,,
2024-11-01,united_states,0.027494,total


In [21]:
import datetime as dt

today_utc = dt.datetime.now(dt.timezone.utc) 
formatted_today_utc = today_utc.strftime('%Y-%m-%d %H:00:00')

In [22]:
cpi_data_df_clean_prepared = prepare_data_for_simulation(cpi_data_df_clean, str(cpi_data_df_clean.index.min()), formatted_today_utc)

price index: DatetimeIndex(['2024-11-01'], dtype='datetime64[ns]', name='date', freq=None)


E:\Projects\encode hackathon\python_scripts\utils.py:286: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  required_dates = pd.date_range(start=start_date, end=end_date, freq='H')
E:\Projects\encode hackathon\python_scripts\utils.py:293: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  price_timeseries.fillna(method='ffill', inplace=True)


In [23]:
cpi_data_df_clean_prepared

,hour,country,value,expenditure
0,2024-11-01 00:00:00,united_states,0.027494,total
1,2024-11-01 01:00:00,united_states,0.027494,total
2,2024-11-01 02:00:00,united_states,0.027494,total
3,2024-11-01 03:00:00,united_states,0.027494,total
4,2024-11-01 04:00:00,united_states,0.027494,total
...,...,...,...,...
1716,2025-01-11 12:00:00,united_states,0.027494,total
1717,2025-01-11 13:00:00,united_states,0.027494,total
1718,2025-01-11 14:00:00,united_states,0.027494,total
1719,2025-01-11 15:00:00,united_states,0.027494,total


In [26]:
for key in interest_rate_dict.keys():
    interest_rate_dict[key] = prepare_data_for_simulation(interest_rate_dict[key], str(interest_rate_dict[key].index.min()), formatted_today_utc)
print(interest_rate_dict)

price index: DatetimeIndex(['2024-12-01'], dtype='datetime64[ns]', name='date', freq=None)
price index: DatetimeIndex(['2024-12-01'], dtype='datetime64[ns]', name='date', freq=None)
price index: DatetimeIndex(['2024-12-01'], dtype='datetime64[ns]', name='date', freq=None)
{'Overnight Interbank Rate':                     hour   value        country
0    2024-12-01 00:00:00  0.0448  united_states
1    2024-12-01 01:00:00  0.0448  united_states
2    2024-12-01 02:00:00  0.0448  united_states
3    2024-12-01 03:00:00  0.0448  united_states
4    2024-12-01 04:00:00  0.0448  united_states
...                  ...     ...            ...
996  2025-01-11 12:00:00  0.0448  united_states
997  2025-01-11 13:00:00  0.0448  united_states
998  2025-01-11 14:00:00  0.0448  united_states
999  2025-01-11 15:00:00  0.0448  united_states
1000 2025-01-11 16:00:00  0.0448  united_states

[1001 rows x 3 columns], '3-Month Rate':                     hour   value        country
0    2024-12-01 00:00:00  0.0446

E:\Projects\encode hackathon\python_scripts\utils.py:286: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  required_dates = pd.date_range(start=start_date, end=end_date, freq='H')
E:\Projects\encode hackathon\python_scripts\utils.py:293: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  price_timeseries.fillna(method='ffill', inplace=True)
E:\Projects\encode hackathon\python_scripts\utils.py:286: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  required_dates = pd.date_range(start=start_date, end=end_date, freq='H')
E:\Projects\encode hackathon\python_scripts\utils.py:293: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  price_timeseries.fillna(method='ffill', inplace=True)
E:\Projects\encode hackathon\python_sc

In [28]:
interest_rate_dict['10-Year Rate']

,hour,value,country
0,2024-12-01 00:00:00,0.0439,united_states
1,2024-12-01 01:00:00,0.0439,united_states
2,2024-12-01 02:00:00,0.0439,united_states
3,2024-12-01 03:00:00,0.0439,united_states
4,2024-12-01 04:00:00,0.0439,united_states
...,...,...,...
996,2025-01-11 12:00:00,0.0439,united_states
997,2025-01-11 13:00:00,0.0439,united_states
998,2025-01-11 14:00:00,0.0439,united_states
999,2025-01-11 15:00:00,0.0439,united_states
